In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression  


In [ ]:
def model_score(y_true, y_pred):
    classify_report = metrics.classification_report(y_true, y_pred)
    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    overall_accuracy = metrics.accuracy_score(y_true, y_pred)
    acc_for_each_class = metrics.precision_score(y_true, y_pred, average=None)
    average_accuracy = np.mean(acc_for_each_class)
    score = metrics.accuracy_score(y_true, y_pred)
    print('classify_report : \n', classify_report)
    print('confusion_matrix : \n', confusion_matrix)
    print('acc_for_each_class : \n', acc_for_each_class)
    print('average_accuracy: {0:f}'.format(average_accuracy))
    print('overall_accuracy: {0:f}'.format(overall_accuracy))
    print('score: {0:f}'.format(score))

In [ ]:
#考虑昨天最后几单bid和ask会影响今天的报价
class DataPre():
    def __init__(self,filename,n_gram):
        self.filename = filename
        self.n_gram = n_gram

    def get_orignal_data(self):
        self.orignal_data = pd.read_csv(self.filename,header=None)
        self.orignal_data = self.orignal_data.groupby([11]).apply(lambda x :x[1:-1])
        self.orignal_data.drop([11],axis=1,inplace=True)
        self.orignal_data = self.orignal_data.reset_index()
        self.orignal_data_x = self.orignal_data[[i for i in range(10)]]
        self.orignal_data_y = self.orignal_data[10]

    def data_split(self):
        data_ = np.concatenate(self.orignal_data_x.values)
        split_data = pd.DataFrame([ data_[i*10:i*10+10*self.n_gram ] for i in range(len(self.orignal_data_x)-self.n_gram+1 )])
        split_data_y = self.orignal_data_y[self.n_gram-1:].values
        split_data['y'] = split_data_y
        return split_data
    

data = DataPre('/content/drive/MyDrive/data.csv',3)
data.get_orignal_data()
split_data = data.data_split()

In [ ]:
#考虑昨天最后几单bid和ask不会影响今天的报价
class DataPre():
    def __init__(self,filename,n_gram):
        self.filename = filename
        self.n_gram = n_gram

    def get_orignal_data(self):
        self.orignal_data = pd.read_csv(self.filename,header=None)
        self.orignal_data = self.orignal_data.groupby([11]).apply(lambda x :x[1:-1])
        self.orignal_data.drop([11],axis=1,inplace=True)
        self.orignal_data = self.orignal_data.reset_index()

    def data_split(self,x):
#         split_index = [[i+j for j in range(self.n_gram)] for i in range(len(self.orignal_data)-self.n_gram+1)]
#         split_data_x = [np.concatenate(data.orignal_data_x.iloc[i].values) for i in split_index]
        orignal_data_x = x[[i for i in range(10)]]
        orignal_data_y = x[10]
        data_ = np.concatenate(orignal_data_x.values)
        split_data = pd.DataFrame([ data_[i*10:i*10+10*self.n_gram ] for i in range(len(orignal_data_x)-self.n_gram+1 )])
        split_data_y = orignal_data_y[self.n_gram-1:].values
        split_data['y'] = split_data_y
        return split_data
    
    def get_data(self):
        result = self.orignal_data.drop(['level_1'],axis=1).groupby([11]).apply(lambda x:self.data_split(x))
        result = pd.DataFrame(result.values)
        result.rename(columns={result.columns[-1]:'y'},inplace=True)
        return result
    
# class Moldes(self)

data = DataPre('/content/drive/MyDrive/data.csv',3)
data.get_orignal_data()
split_data = data.get_data()

In [ ]:
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(split_data.drop(['y'],axis=1), split_data['y'], test_size=test_size, random_state=seed)
# X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train , test_size=0.2, random_state=seed)

In [ ]:
#xgboost 模型效果
model = XGBClassifier(max_depth=3, learning_rate=0.01, n_estimators=50, silent=True, objective="binary:logistic", booster='gbtree')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model_score(y_test,y_pred)


classify_report : 
               precision    recall  f1-score   support

         0.0       0.58      0.93      0.71     55237
         1.0       0.63      0.15      0.24     43843

    accuracy                           0.58     99080
   macro avg       0.60      0.54      0.48     99080
weighted avg       0.60      0.58      0.50     99080

confusion_matrix : 
 [[51330  3907]
 [37289  6554]]
acc_for_each_class : 
 [0.57922116 0.62651754]
average_accuracy: 0.602869
overall_accuracy: 0.584215
score: 0.584215
